In [ ]:
import logging
import re
import ssl
import time
from datetime import date
from functools import reduce

import numpy as np
import pandas as pd
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.web.slack_response import SlackResponse
from sqlalchemy import MetaData, Table, Selectable
from sqlalchemy.engine import Engine
from sqlalchemy.sql import and_, case, func, or_, select, text

from pax_achievements import (
    cadre,
    el_presidente,
    hammer_not_nail,
    hdtf,
    leader_of_men,
    nation_sql,
    posts,
    region_sql,
    six_pack,
    the_boss,
    the_monk,
    the_priest,
award_list,
award_view
)
from utils import mysql_connection, send_to_slack


In [ ]:
logging.basicConfig(format="%(asctime)s [%(levelname)s]:%(message)s",
                        level=logging.INFO,
                        datefmt="%Y-%m-%d %H:%M:%S")

In [ ]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
%time metadata.reflect(engine, schema="weaselbot")

In [ ]:
%time df_regions = pd.read_sql(region_sql(metadata), engine)
%time nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

In [ ]:
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source")

dfs = []
############# Q Source ##############
dfs.append(the_priest(nation_df, bb_filter, ao_filter))
dfs.append(the_monk(nation_df, bb_filter, ao_filter))
############### END #################

# For beatdowns, we want to exclude QSource and Ruck (blackops too? What is blackops?)
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s|ruck")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source|ruck")

############ ALL ELSE ###############
dfs.append(leader_of_men(nation_df, bb_filter, ao_filter))
dfs.append(the_boss(nation_df, bb_filter, ao_filter))
dfs.append(hammer_not_nail(nation_df, bb_filter, ao_filter))
dfs.append(cadre(nation_df, bb_filter, ao_filter))
dfs.append(el_presidente(nation_df, bb_filter, ao_filter))

s = posts(nation_df, bb_filter, ao_filter)
for val in [25, 50, 100, 150, 200]:
    dfs.append(s[s.ao_id >= val].reset_index().drop('ao_id', axis=1).rename({"date": "date_awarded"}, axis=1))

dfs.append(six_pack(nation_df, bb_filter, ao_filter))
dfs.append(hdtf(nation_df, bb_filter, ao_filter))

In [ ]:
df_regions.paxminer_schema.unique()

In [ ]:
for r in df_regions.itertuples(index=False):
    if r.paxminer_schema == "f3noho":
        row = r

`f3spiritofstl` changed their awards table. They took out "the_priest" and a few others. ??? That's fine. Do we want that? 

We're now in the loop of all regions. Pick one from the above list and run `send_to_slack` to see what's already up-to-date and what new records would be added.

In [ ]:
%time awarded = pd.read_sql(award_view(row, engine, metadata, year), engine, parse_dates=['date_awarded', 'created', 'updated'])
%time awards = pd.read_sql(award_list(row, engine, metadata), engine)

In [ ]:
send_to_slack(row, year, awarded, awards, dfs)

In [ ]:
awards